In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
#based on https://www.kaggle.com/code/pankul/image-classification-w-vgg16-weights
IMAGE_SIZE = [224, 224]  # we will keep the image size as (64,64). You can increase the size for better results. 
batch_size = 64
nb_epochs = 10
train_data_dir = '../input/fruits/fruits-360_dataset/fruits-360/Training'
test_data_dir = '../input/fruits/fruits-360_dataset/fruits-360/Test'

# getting the number of classes i.e. type of fruits
folders = glob(train_data_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

train_ds = tf.keras.utils.image_dataset_from_directory(
  train_data_dir,
  image_size=IMAGE_SIZE,
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_data_dir,
  image_size=IMAGE_SIZE,
  batch_size=batch_size)

datagen = ImageDataGenerator(rescale=1./255,   # all pixel values will be between 0 an 1
                                    preprocessing_function=preprocess_input)

train_gen = datagen.flow_from_directory(train_data_dir, target_size = IMAGE_SIZE, batch_size = batch_size, class_mode = 'categorical')
test_gen = datagen.flow_from_directory(test_data_dir, target_size = IMAGE_SIZE, batch_size = batch_size, class_mode = 'categorical')

In [ ]:
for data_batch, labels_batch in train_ds:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

In [ ]:
# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg = VGG19(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  # input_shape = (64,64,3) as required by VGG

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)
x = Dense(128, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
x = Dense(num_classes, activation = 'softmax')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg.input, outputs = x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
earlystop = EarlyStopping(
    monitor='val_acc',
    min_delta=0.005,
    patience=5,
    verbose=1,
)

In [ ]:
history = model.fit(train_gen,  
                   epochs = 30,  
                   validation_data = test_gen,
                   callbacks=[earlystop,]) 

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()